<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jjc/NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
datas = []
data = ['me free lottery', 'spam']
datas.append(data)
data = ['free get free you', 'spam']
datas.append(data)
data = ['you free scholarship', 'normal']
datas.append(data)
data = ['free to contact me', 'normal']
datas.append(data)
data = ['you won award', 'normal']
datas.append(data)
data = ['you ticket lottery', 'spam']
datas.append(data)

datas

In [352]:
import pandas as pd
import numpy as np
# 데이터프레임은 인덱스에 단어, 칼럼에 타겟
class NBC():
    
    df = pd.DataFrame([])
    k = 0.5

    def __init__(self, **kwargs):
        try:
            self.k = kwargs['k']
        except:
            pass
    
    def add_data(self, datas):
        a_name = ''
        b_name = ''
        a_word = []
        b_word = []

        for data in datas:
            if a_name == '':
                a_name = data[1]
            if b_name == '':
                if data[1] != a_name :
                    b_name = data[1]
                
            if data[1] == a_name:
                a_word.extend(data[0].split())
            else:
                b_word.extend(data[0].split())

        #extend는 inplace 효과 있음
        t_word = []
        t_word.extend(a_word)
        t_word.extend(b_word)
        set_word = set(t_word)
        a_count = []
        b_count = []
        for sw in set_word:
            a_count.append(a_word.count(sw))
            b_count.append(b_word.count(sw))
        
        final_data = list(zip(set_word, np.zeros(len(set_word)), np.zeros(len(set_word))))
        df = pd.DataFrame(final_data, columns = ['words', a_name, b_name])
        df.set_index('words', inplace = True)
        df[a_name] = a_count
        df[b_name] = b_count
        self.df = df
        return self.df

    def lp_sm(a, b):
        return (0.5 + a) / (1 + b)
    
    def pol_score(self, target, s):
        df = self.df
        tokens = s.split()
        a_total = df[target].sum()
        total = df.sum().sum()
        b_total = total - a_total

        A = np.log(float(a_total) / float(total))
        B = np.log(float(b_total) / float(total))

        for token in tokens:
            A += np.log( (0.5 + df[target][token]) / ( 1 + a_total) )
            B += np.log( (0.5 + df[df.index == token].sum().sum() - df[target][token]) / (1 + b_total) ) 

        return np.exp(A) / (np.exp(A) + np.exp(B))

In [353]:
nbc = NBC()
nbc.add_data(datas)

,spam,normal
words,,
scholarship,0,1
free,3,2
get,1,0
ticket,1,0
lottery,2,0
contact,0,1
you,2,2
won,0,1
award,0,1


In [354]:
nbc.pol_score('spam', 'free lottery')

0.8749999999999999